# Kriptografi GOST

### Penyimpanan Kunci
Algoritma GOST menerima kunci sebanyak 256 bit atau setara 32 karakter (byte)  
Kunci tersebut kemudian dibagi lagi menjadi 8 bagian (K0-K7) dengan panjang masing-masing 4 byte

In [56]:
# 32 byte = 32 karakter
kunci = input("Masukkan kunci (256 bit/32 byte): ")

# Jika panjang kunci kurang dari 32
if len(kunci) < 32:
    # Gunakan auto-key untuk memperpanjang kunci (ceil division)
    # Ceil division diemulasikan melalui floor division + 1
    kunci = kunci * (32 // len(kunci) + 1)

# Cukup ambil 32 karakter pertama
kunci = kunci[:32]

print("Kunci (32 byte):", kunci)

Kunci (32 byte): abcabcabcabcabcabcabcabcabcabcab


In [57]:
# Bagi kunci menjadi 8 sub-kunci (K0-K7)
# Panjang tiap kunci 32 byte / 8 = 4 byte

pjg_kunci = 4
# Bagi kunci berdasarkan panjang string (4 karakter)
# https://stackoverflow.com/q/13673060
kunci = [ kunci[i:pjg_kunci+i] for i in range(0, len(kunci), pjg_kunci) ]

print("Kunci (8x4 byte):", kunci)

Kunci (8x4 byte): ['abca', 'bcab', 'cabc', 'abca', 'bcab', 'cabc', 'abca', 'bcab']


In [58]:
def teks_ke_biner(teks, spasi = " ", bit = 8):
    # Konversi teks dari ASCII menjadi X bit biner
    # https://stackoverflow.com/q/18815820, https://stackoverflow.com/a/47664902
    return spasi.join(f"{ord(i):0{bit}b}" for i in teks)

kunci_biner = [ teks_ke_biner(i) for i in kunci ]
print("Kunci dalam biner:", kunci_biner)

Kunci dalam biner: ['01100001 01100010 01100011 01100001', '01100010 01100011 01100001 01100010', '01100011 01100001 01100010 01100011', '01100001 01100010 01100011 01100001', '01100010 01100011 01100001 01100010', '01100011 01100001 01100010 01100011', '01100001 01100010 01100011 01100001', '01100010 01100011 01100001 01100010']


In [59]:
# Balik posisi biner kunci (bit paling kiri menjadi paling kanan, dan sebaliknya)
kunci_biner = [ i[::-1] for i in kunci_biner ]
print("Kunci dalam biner (setelah dibalik):", kunci_biner)

Kunci dalam biner (setelah dibalik): ['10000110 11000110 01000110 10000110', '01000110 10000110 11000110 01000110', '11000110 01000110 10000110 11000110', '10000110 11000110 01000110 10000110', '01000110 10000110 11000110 01000110', '11000110 01000110 10000110 11000110', '10000110 11000110 01000110 10000110', '01000110 10000110 11000110 01000110']


### Penyimpanan Teks
Algoritma GOST merupakan block cipher yang menerima plainteks 64 bit atau 8 karakter (byte)  
Bila plainteks lebih panjang dari itu, maka akan dipecah menjadi lebih dari 1 blok

In [60]:
# 8 byte = 8 karakter
plainteks = input("Masukkan plainteks (64 bit/8 byte): ")

# Bagi plainteks menjadi beberapa blok (jika lebih dari 8 byte)
# Tiap blok terdiri dari 8 karakter (byte)

pjg_plainteks = 8
# Bagi plainteks berdasarkan panjang string (8 karakter)
# https://stackoverflow.com/q/13673060
plainteks = [ plainteks[i:pjg_plainteks+i] for i in range(0, len(plainteks), pjg_plainteks) ]
print("Plainteks (8 byte per blok):", plainteks)

Plainteks (8 byte per blok): ['andhika ', 'wibawa b', 'hagaskar', 'a']


In [61]:
# Jika panjang suatu blok kurang dari 8 byte
# Tambahkan padding berupa kode ASCII 0 (null) di depan

for i in range(len(plainteks)):
    if len(plainteks[i]) < pjg_plainteks:
        plainteks[i] = ("\0" * (pjg_plainteks - len(plainteks[i]))) + plainteks[i]

print("Plainteks (per blok, dengan padding):", plainteks)

Plainteks (per blok, dengan padding): ['andhika ', 'wibawa b', 'hagaskar', '\x00\x00\x00\x00\x00\x00\x00a']


In [62]:
# Konversi plainteks ke biner
plainteks_biner = [ teks_ke_biner(i) for i in plainteks ]

print("Plainteks dalam biner (per blok):")
for i in range(len(plainteks_biner)):
    print(f"Plainteks {i} -", f"['{plainteks_biner[i]}']")

Plainteks dalam biner (per blok):
Plainteks 0 - ['01100001 01101110 01100100 01101000 01101001 01101011 01100001 00100000']
Plainteks 1 - ['01110111 01101001 01100010 01100001 01110111 01100001 00100000 01100010']
Plainteks 2 - ['01101000 01100001 01100111 01100001 01110011 01101011 01100001 01110010']
Plainteks 3 - ['00000000 00000000 00000000 00000000 00000000 00000000 00000000 01100001']


In [63]:
# Bagi tiap blok plainteks menjadi 2 bagian (yaitu R0 dan L0)
# Tiap blok berukuran 8 byte, maka R0 = byte-0 sampai byte-3, L0 = byte-4 sampai byte-8

for i in range(len(plainteks_biner)):
    # Pecah biner plainteks ke dalam list berdasarkan spasi
    tmp_teks = plainteks_biner[i].split()
    # Gabung kembali menjadi 2 bagian terpisah (R0 dan L0)
    plainteks_biner[i] = [ " ".join(tmp_teks[0:4]),  " ".join(tmp_teks[4:8]) ]

print("Plainteks R0 dan L0 dalam biner (per blok):")
for i in range(len(plainteks_biner)):
    print(f"Plainteks {i} -", plainteks_biner[i])

Plainteks R0 dan L0 dalam biner (per blok):
Plainteks 0 - ['01100001 01101110 01100100 01101000', '01101001 01101011 01100001 00100000']
Plainteks 1 - ['01110111 01101001 01100010 01100001', '01110111 01100001 00100000 01100010']
Plainteks 2 - ['01101000 01100001 01100111 01100001', '01110011 01101011 01100001 01110010']
Plainteks 3 - ['00000000 00000000 00000000 00000000', '00000000 00000000 00000000 01100001']


In [64]:
# Balik posisi biner plainteks (bit paling kiri menjadi paling kanan, dan sebaliknya)
for i in range(len(plainteks_biner)):
    for j in range(len(plainteks_biner[i])):
        plainteks_biner[i][j] = plainteks_biner[i][j][::-1]

print("Plainteks R0 dan L0 dalam biner (per blok, setelah dibalik):")
for i in range(len(plainteks_biner)):
    print(f"Plainteks {i} -", plainteks_biner[i])

Plainteks R0 dan L0 dalam biner (per blok, setelah dibalik):
Plainteks 0 - ['00010110 00100110 01110110 10000110', '00000100 10000110 11010110 10010110']
Plainteks 1 - ['10000110 01000110 10010110 11101110', '01000110 00000100 10000110 11101110']
Plainteks 2 - ['10000110 11100110 10000110 00010110', '01001110 10000110 11010110 11001110']
Plainteks 3 - ['00000000 00000000 00000000 00000000', '10000110 00000000 00000000 00000000']


### Fungsi dan Variabel Bantuan

S-box digunakan untuk subtitusi nilai biner masukan menjadi nilai biner lainnya  
Keamanan algoritma GOST sangat bergantung kepada isi dari S-box

In [65]:
# GOST R 34.12-2015 S-Box
SBOX = [
    # 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15
    (12, 4, 6, 2, 10, 5, 11, 9, 14, 8, 13, 7, 0, 3, 15, 1),
    (6, 8, 2, 3, 9, 10, 5, 12, 1, 14, 4, 7, 11, 13, 0, 15),
    (11, 3, 5, 8, 2, 15, 10, 13, 14, 1, 7, 4, 12, 9, 6, 0),
    (12, 8, 2, 1, 13, 4, 15, 6, 7, 0, 10, 5, 3, 14, 9, 11),
    (7, 15, 5, 10, 8, 1, 6, 13, 0, 9, 3, 14, 11, 4, 2, 12),
    (5, 13, 15, 6, 9, 2, 12, 10, 11, 7, 8, 1, 4, 3, 14, 0),
    (8, 14, 2, 5, 6, 9, 1, 12, 15, 4, 11, 0, 13, 10, 3, 7),
    (1, 7, 14, 13, 0, 5, 8, 3, 4, 15, 10, 6, 9, 12, 11, 2)
]

"""
    Contoh Input:
        Biner: 1111 0000 0001 0011 ...
               (SBOX0:15 SBOX1:0 SBOX2:1 SBOX3:3)
               (SBOX sebagai baris, nilai sebagai kolom)
    Contoh Output:
        Biner: 0001 0110 0011 0001 ...
               (SBOX0:1 SBOX1:6 SBOX2:3 SBOX3:1)
"""
def substitusi_sbox(biner):
    # Hilangkan spasi pada biner bila ada
    biner = biner.replace(" ", "")

    # Pecah biner per 4 bit (rentang nilai 0-15) untuk diproses ke S-box
    biner = [ biner[i:4+i] for i in range(0, len(biner), 4) ]
    # Substitusi (ubah) nilai biner saat ini menjadi nilai baru
    for i in range(len(biner)):
        nilai = biner_ke_angka(biner[i], 4)
        biner[i] = angka_ke_biner(SBOX[i][nilai], 4)
    
    # Kembalikan biner dalam bentuk string (bukan list)
    return "".join(biner)

Fungsi-fungsi biner digunakan untuk konversi biner ke/dari tipe data lainnya

In [66]:
def biner_ke_teks(biner, bit = 8, padding = True):
    # Hilangkan spasi pada biner bila ada
    biner = biner.replace(" ", "")

    # Konversi biner ke teks berdasarkan jumlah bit
    # https://stackoverflow.com/a/40559005
    teks = "".join(
        chr(int(biner[i*bit : i*bit+bit], 2)) for i in range(len(biner) // bit)
    )
    # Jika tanpa padding, maka hapus kode ASCII 0 (null) dari teks
    if not padding: teks = teks.replace("\0", "")

    return teks

In [67]:
def biner_ke_angka(biner, bit = 32):
    # Hilangkan spasi pada biner bila ada
    biner = biner.replace(" ", "")

    # Konversi biner ke angka berdasarkan jumlah bit
    # Modifikasi dari fungsi biner ke teks sebelumnya
    if len(biner) // bit > 1:
        return [ int(biner[i*bit : i*bit+bit], 2) for i in range(len(biner) // bit) ]
    else:
        return int(biner[:bit], 2)

def angka_ke_biner(angka, bit = 32):
    # Konversi angka ke biner berdasarkan jumlah bit
    # https://stackoverflow.com/q/699866
    return format(angka, f"0{bit}b")

Rotate left shift merupakan merupakan fungsi pergeseran digit biner

In [68]:
"""
    Contoh Input:
        Biner = "11111111 11100000 00000000 00000000"
        Bit = 11
    Contoh Output:
        Biner = "00000000 00000000 00000111 11111111"
        (11 bit di kiri sekarang menjadi di kanan)
"""
def rotate_left_shift(biner, bit = 11):
    biner = biner.replace(" ", "")
    return biner[bit:] + biner[:bit]

### Proses Enkripsi
Proses enkripsi dalam GOST terdiri dari 32 round (0-31)  
Ronde 0-7, 8-15, 16-23, dan 24-31 digunakan kunci K0-K7  
Ronde 24-32 digunakan kunci K7-K0 (berkebalikan)

In [69]:
# Proses enkripsi GOST (hanya untuk 1 blok plainteks)
# Eksekusi fungsi enkripsi kembali bila jumlah blok lebih dari 1

def enkripsi(plainteks_biner, kunci_biner):
    # Inisiasi RX dan LX (0-31) untuk tiap round nantinya
    # Ambil R0 dan L0 untuk digunakan pertama kali (spasi dihilangkan)
    # R1, L1, dan seterusnya baru akan dibentuk setelah R0 dan L0 dipakai
    R = [ plainteks_biner[0].replace(" ", "") ]
    L = [ plainteks_biner[1].replace(" ", "") ]

    # Kunci (K0-K7) dimana spasi dalam digit biner telah dihilangkan
    K = [ i.replace(" ", "") for i in kunci_biner ]

    # j = Indeks kunci yang sedang digunakan (K0-K7)
    # i = Putaran (round) saat ini

    j = 0
    for i in range(32):
        if i < 24:
            print(f"R{i}: {R[i]}, L{i}: {L[i]}, K{j}: {K[j]}")
            # Tambahkan nilai RX dan KX kemudian modulus dengan 2^32
            hasil = (biner_ke_angka(R[i]) + biner_ke_angka(K[j])) % (2 ** 32)
            print(f"R{i} + K{j} % 2^32: {angka_ke_biner(hasil)}")
            # Ulang indeks kunci tiap 7 putaran (K0-K7) selama masih dibawah 24 putaran
            if j == 7: j = 0
            else: j = j + 1
        else:
            # Jika sudah 24 putaran maka indeks kunci dibalik (dari K7-K0)
            if i == 24: j = 7
            print(f"R{i}: {R[i]}, L{i}: {L[i]}, K{j}: {K[j]}")
            # Tambahkan nilai RX dan KX kemudian modulus dengan 2^32
            hasil = (biner_ke_angka(R[i]) + biner_ke_angka(K[j])) % (2 ** 32)
            print(f"R{i} + K{j} % 2^32: {angka_ke_biner(hasil)}")
            j = j - 1
        
        # Konversi biner ke angka
        hasil_biner = angka_ke_biner(hasil)
        # Substitusi (ubah) biner sesuai isi S-box
        hasil_biner = substitusi_sbox(hasil_biner)
        print(f"Hasil S-box: {hasil_biner}")
        
        # Langkah 2
        if i < 31:
            # Hasilkan RX+1 untuk perulangan selanjutnya
            R.append(
                # RLS hasil biner kemudian XOR-kan dengan LX (X = round saat ini)
                angka_ke_biner(int(rotate_left_shift(hasil_biner), 2) ^ int(L[i], 2))
            )
            # Hasilkan LX+1 untuk perulangan selanjutnya
            L.append(R[i])

            print(f"RLS S-box: {angka_ke_biner(int(rotate_left_shift(hasil_biner), 2))}")
            print(f"RLS S-box XOR L{i}: {angka_ke_biner(int(rotate_left_shift(hasil_biner), 2) ^ int(L[i], 2))}\n")
        else:
            # Hasilkan RX+1 (R32)
            R.append(R[i])
            # Hasilkan LX+1 (L32)
            L.append(
                # RLS hasil biner kemudian XOR-kan dengan LX (X = round saat ini)
                angka_ke_biner(int(rotate_left_shift(hasil_biner), 2) ^ int(L[i], 2))
            )

            print(f"RLS S-box: {angka_ke_biner(int(rotate_left_shift(hasil_biner), 2))}")
            print(f"RLS S-box XOR L{i}: {angka_ke_biner(int(rotate_left_shift(hasil_biner), 2) ^ int(L[i], 2))}")

            # Balik posisi bit kedua variabel kemudian hasilkan cipherteks
            R[i+1] = R[i+1][::-1]
            L[i+1] = L[i+1][::-1]
            return R[i+1] + L[i+1]

cipherteks_biner = []
for i in plainteks_biner:
    # Bila plainteks lebih dari 1 blok, maka enkripsi berkali-kali sesuai jumlah blok
    cipherteks_biner.append(enkripsi(i, kunci_biner))
    print(f"Hasil cipher (1 blok): {biner_ke_teks(cipherteks_biner[-1])} ({cipherteks_biner[-1]})\n")

cipherteks = "".join([ biner_ke_teks(i) for i in cipherteks_biner ])

if len(cipherteks_biner) > 1:
    print(f"Hasil akhir cipher (semua blok): {[ biner_ke_teks(i) for i in cipherteks_biner ]}")
    print(f"Hasil akhir cipher (semua blok, dalam biner): {cipherteks_biner}")

R0: 00010110001001100111011010000110, L0: 00000100100001101101011010010110, K0: 10000110110001100100011010000110
R0 + K0 % 2^32: 10011100111011001011110100001100
Hasil S-box: 10001011011000111110001110001001
RLS S-box: 00011111000111000100110001011011
RLS S-box XOR L0: 00011011100110101001101011001101

R1: 00011011100110101001101011001101, L1: 00010110001001100111011010000110, K1: 01000110100001101100011001000110
R1 + K1 % 2^32: 01100010001000010110000100010011
Hasil S-box: 10110010010110000110110111101101
RLS S-box: 11000011011011110110110110010010
RLS S-box XOR L1: 11010101010010010001101100010100

R2: 11010101010010010001101100010100, L2: 00011011100110101001101011001101, K2: 11000110010001101000011011000110
R2 + K2 % 2^32: 10011011100011111010000111011010
Hasil S-box: 10000111111010110011110110101010
RLS S-box: 01011001111011010101010000111111
RLS S-box XOR L2: 01000010011101111100111011110010

R3: 01000010011101111100111011110010, L3: 11010101010010010001101100010100, K3: 10000110

### Proses Dekripsi
Proses dekripsi dalam GOST terdiri dari 32 round (0-31)  
Ronde 0-7, 8-15, 16-23, dan 24-31 digunakan kunci K7-K0  
Ronde 24-32 digunakan kunci K0-K7 (berkebalikan)

In [70]:
# Bagi tiap blok cipherteks sebelumnya menjadi 2 bagian (yaitu R0 dan L0)
# Tiap blok berukuran 8 byte, maka R0 = byte-0 sampai byte-3, L0 = byte-4 sampai byte-8

for i in range(len(cipherteks_biner)):
    cipherteks_biner[i] = [ cipherteks_biner[i][:32],  cipherteks_biner[i][32:] ]

print("Cipherteks R0 dan L0 dalam biner (per blok):")
for i in range(len(cipherteks_biner)):
    print(f"Cipherteks {i} -", cipherteks_biner[i])

print("\nCipherteks R0 dan L0 dalam biner (per blok, setelah dibalik):")
for i in range(len(cipherteks_biner)):
    # Balik posisi bit cipherteks (bit paling kanan menjadi paling kiri, dan sebaliknya)
    for j in range(len(cipherteks_biner[i])):
        cipherteks_biner[i][j] = cipherteks_biner[i][j][::-1]
    print(f"Cipherteks {i} -", cipherteks_biner[i])

Cipherteks R0 dan L0 dalam biner (per blok):
Cipherteks 0 - ['01111001101010111111000100110011', '11100110000011110101001000110000']
Cipherteks 1 - ['10010100101001010000000110001101', '00011101010110101110001001110110']
Cipherteks 2 - ['01000001000000000000100011100000', '01110100000110010001100111100111']
Cipherteks 3 - ['01011011010001011011001101111101', '10011010001101100001111000110100']

Cipherteks R0 dan L0 dalam biner (per blok, setelah dibalik):
Cipherteks 0 - ['11001100100011111101010110011110', '00001100010010101111000001100111']
Cipherteks 1 - ['10110001100000001010010100101001', '01101110010001110101101010111000']
Cipherteks 2 - ['00000111000100000000000010000010', '11100111100110001001100000101110']
Cipherteks 3 - ['10111110110011011010001011011010', '00101100011110000110110001011001']


In [71]:
# Proses dekripsi GOST (hanya untuk 1 blok plainteks)
# Eksekusi fungsi dekripsi kembali bila jumlah blok lebih dari 1

def dekripsi(cipherteks_biner, kunci_biner):
    # Inisiasi RX dan LX (0-31) untuk tiap round nantinya
    # Ambil R0 dan L0 untuk digunakan pertama kali (spasi dihilangkan)
    # R1, L1, dan seterusnya baru akan dibentuk setelah R0 dan L0 dipakai
    R = [ cipherteks_biner[0].replace(" ", "") ]
    L = [ cipherteks_biner[1].replace(" ", "") ]

    # Kunci (K0-K7) dimana spasi dalam digit biner telah dihilangkan
    K = [ i.replace(" ", "") for i in kunci_biner ]

    # j = Indeks kunci yang sedang digunakan (K0-K7)
    # i = Putaran (round) saat ini

    j = 0
    for i in range(32):
        if i < 8:
            print(f"R{i}: {R[i]}, L{i}: {L[i]}, K{j}: {K[j]}")
            # Tambahkan nilai RX dan KX kemudian modulus dengan 2^32
            hasil = (biner_ke_angka(R[i]) + biner_ke_angka(K[j])) % (2 ** 32)
            print(f"R{i} + K{j} % 2^32: {angka_ke_biner(hasil)}")
            # Urutan indeks kunci K0-K7 selama dibawah 8 putaran
            if j < 7: j = j + 1
        else:
            print(f"R{i}: {R[i]}, L{i}: {L[i]}, K{j}: {K[j]}")
            # Tambahkan nilai RX dan KX kemudian modulus dengan 2^32
            hasil = (biner_ke_angka(R[i]) + biner_ke_angka(K[j])) % (2 ** 32)
            print(f"R{i} + K{j} % 2^32: {angka_ke_biner(hasil)}")
            # Ulangi indeks kunci dari K7-K0 bila diatas 8 putaran
            if j == 0: j = 7
            else: j = j - 1
        
        # Konversi biner ke angka
        hasil_biner = angka_ke_biner(hasil)
        # Substitusi (ubah) biner sesuai isi S-box
        hasil_biner = substitusi_sbox(hasil_biner)
        print(f"Hasil S-box: {hasil_biner}")
        
        # Langkah 2
        if i < 31:
            # Hasilkan RX+1 untuk perulangan selanjutnya
            R.append(
                # RLS hasil biner kemudian XOR-kan dengan LX (X = round saat ini)
                angka_ke_biner(int(rotate_left_shift(hasil_biner), 2) ^ int(L[i], 2))
            )
            # Hasilkan LX+1 untuk perulangan selanjutnya
            L.append(R[i])

            print(f"RLS S-box: {angka_ke_biner(int(rotate_left_shift(hasil_biner), 2))}")
            print(f"RLS S-box XOR L{i}: {angka_ke_biner(int(rotate_left_shift(hasil_biner), 2) ^ int(L[i], 2))}\n")
        else:
            # Hasilkan RX+1 (R32)
            R.append(R[i])
            # Hasilkan LX+1 (L32)
            L.append(
                # RLS hasil biner kemudian XOR-kan dengan LX (X = round saat ini)
                angka_ke_biner(int(rotate_left_shift(hasil_biner), 2) ^ int(L[i], 2))
            )

            print(f"RLS S-box: {angka_ke_biner(int(rotate_left_shift(hasil_biner), 2))}")
            print(f"RLS S-box XOR L{i}: {angka_ke_biner(int(rotate_left_shift(hasil_biner), 2) ^ int(L[i], 2))}")

            # Balik posisi bit kedua variabel kemudian hasilkan plainteks
            R[i+1] = R[i+1][::-1]
            L[i+1] = L[i+1][::-1]
            return R[i+1] + L[i+1]

decipherteks_biner = []
for i in cipherteks_biner:
    # Bila cipherteks lebih dari 1 blok, maka dekripsi berkali-kali sesuai jumlah blok
    decipherteks_biner.append(dekripsi(i, kunci_biner))
    print(f"Hasil decipherteks (1 blok): {biner_ke_teks(decipherteks_biner[-1])} ({decipherteks_biner[-1]})\n")

if len(decipherteks_biner) > 1:
    print(f"Hasil akhir decipherteks (semua blok): {[ biner_ke_teks(i, padding = False) for i in decipherteks_biner ]}")
    print(f"Hasil akhir decipherteks (semua blok, dalam biner): {decipherteks_biner}")

R0: 11001100100011111101010110011110, L0: 00001100010010101111000001100111, K0: 10000110110001100100011010000110
R0 + K0 % 2^32: 01010011010101100001110000100100
Hasil S-box: 01010011111111111111010000100000
RLS S-box: 11111111101000010000001010011111
RLS S-box XOR L0: 11110011111010111111001011111000

R1: 11110011111010111111001011111000, L1: 11001100100011111101010110011110, K1: 01000110100001101100011001000110
R1 + K1 % 2^32: 00111010011100101011100100111110
Hasil S-box: 00100100110100101110011101011011
RLS S-box: 10010111001110101101100100100110
RLS S-box XOR L1: 01011011101101010000110010111000

R2: 01011011101101010000110010111000, L2: 11110011111010111111001011111000, K2: 11000110010001101000011011000110
R2 + K2 % 2^32: 00100001111110111001001101111110
Hasil S-box: 01101000000001011001011011001011
RLS S-box: 00101100101101100101101101000000
RLS S-box XOR L2: 11011111010111011010100110111000

R3: 11011111010111011010100110111000, L3: 01011011101101010000110010111000, K3: 10000110